In [29]:
import streamlit as st
import streamlit.components.v1 as components
import csv
import string
import os
import pickle
import pandas as pd
from datetime import datetime as dt
from base64 import b64encode
#import proximitypyhash as pph
#import pygeohash as gh
from streamlit_folium import st_folium
from streamlit_folium import folium_static
import folium
import numpy as np

from agile.filtering import query_location, query_date, query_adid, query_node
from agile.mapping import data_map 
from agile.locations import locations_of_interest
from agile.people import colocation
from agile.prediction import double_cluster, get_top_N_clusters, haversine
from agile.utils.tag import find_all_nearby_nodes
from agile.utils.geocode import reverse_geocode
from agile.utils.files import find, random_line, save, random_name
from agile.utils.dataframes import modify_and_sort_columns
from agile.profile import Profile
from agile.samsreport import Report
from agile.centrality import compute_top_centrality
from agile.overview import adid_value_counts

from streamlit_option_menu import option_menu
import pygeohash as gh

In [30]:
with open('saved_data/saved_df.pkl', 'rb') as file:
    df = pickle.load(file)

In [38]:
adid = '21191f02-c141-e3f6-901e-f0c9222be423'
num_clusters = 4

cluster_data = double_cluster(adid, df)
loi_data = get_top_N_clusters(cluster_data, num_clusters)
loi_data_i = pd.DataFrame()
loi_data

loi_data['iter'] = [0] * len(loi_data)

In [59]:
def filter_close_coordinates(df, threshold_distance=0.13):
    new_df = pd.DataFrame(columns=df.columns)

    for i, row1 in df.iterrows():
        keep_row = True
        for _, row2 in new_df.iterrows():
            distance = haversine(row1['latitude'], row1['longitude'], row2['latitude'], row2['longitude'])
            if distance < threshold_distance:
                keep_row = False
                break
        if keep_row:
            new_df = new_df.append(row1, ignore_index=True)

    return new_df

In [67]:
cluster_data = pd.DataFrame()
loi_data = pd.DataFrame()
loi_data_i = pd.DataFrame()

for i in range(1,5):
    print(i)
    cluster_data = double_cluster(adid, df)
    loi_data_i = get_top_N_clusters(cluster_data, i)
    loi_data_i['i'] = [i] * len(loi_data_i)
    loi_data_i = filter_close_coordinates(loi_data_i, threshold_distance=.7)
    loi_data = pd.concat([loi_data, loi_data_i])
    
for i in range(1,25,3):
    for j in range(73,0,-4):
        print(i,j)
        loi_data = pd.concat([loi_data,locations_of_interest(df, adid, i, j)]).reset_index(drop=True)        

loi_data = loi_data.drop_duplicates()        
    
loi_data

1


/tmp/ipykernel_8669/2294503313.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row1, ignore_index=True)


2


/tmp/ipykernel_8669/2294503313.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row1, ignore_index=True)


3


/tmp/ipykernel_8669/2294503313.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row1, ignore_index=True)


4


/tmp/ipykernel_8669/2294503313.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row1, ignore_index=True)


1 73
1 69
1 65
1 61
1 57
1 53
1 49
1 45
1 41
1 37
1 33
1 29
1 25
1 21
1 17
1 13
1 9
1 5
1 1
4 73
4 69
4 65
4 61
4 57
4 53
4 49
4 45
4 41
4 37
4 33
4 29
4 25
4 21
4 17
4 13
4 9
4 5
4 1
7 73
7 69
7 65
7 61
7 57
7 53
7 49
7 45
7 41
7 37
7 33
7 29
7 25
7 21
7 17
7 13
7 9
7 5
7 1
10 73
10 69
10 65
10 61
10 57
10 53
10 49
10 45
10 41
10 37
10 33
10 29
10 25
10 21
10 17
10 13
10 9
10 5
10 1
13 73
13 69
13 65
13 61
13 57
13 53
13 49
13 45
13 41
13 37
13 33
13 29
13 25
13 21
13 17
13 13
13 9
13 5
13 1
16 73
16 69
16 65
16 61
16 57
16 53
16 49
16 45
16 41
16 37
16 33
16 29
16 25
16 21
16 17
16 13
16 9
16 5
16 1
19 73
19 69
19 65
19 61
19 57
19 53
19 49
19 45
19 41
19 37
19 33
19 29
19 25
19 21
19 17
19 13
19 9
19 5
19 1
22 73
22 69
22 65
22 61
22 57
22 53
22 49
22 45
22 41
22 37
22 33
22 29
22 25
22 21
22 17
22 13
22 9
22 5
22 1


,latitude,longitude,datetime,geohash,advertiser_id,label,i
0,46.221791,-119.031120,2018-04-08 08:27:15.500,c25qgqkjmy,21191f02-c141-e3f6-901e-f0c9222be423,297,1
1,46.168194,-118.953881,2018-04-08 08:42:25.000,c25qthsmjx,21191f02-c141-e3f6-901e-f0c9222be423,300,2
2,46.226192,-119.037492,2018-04-08 08:26:00.500,c25qgppx16,21191f02-c141-e3f6-901e-f0c9222be423,287,2
3,46.137972,-118.752535,2018-04-08 09:19:00.500,c25w3xpj1m,21191f02-c141-e3f6-901e-f0c9222be423,299,3
4,46.138450,-118.806781,2018-04-08 09:09:58.000,c25w2rpxcq,21191f02-c141-e3f6-901e-f0c9222be423,147,3
5,46.199828,-118.999448,2018-04-08 08:33:01.000,c25qu47jxb,21191f02-c141-e3f6-901e-f0c9222be423,20,3
6,46.212748,-119.018090,2018-04-08 08:29:34.500,c25qgsv8v2,21191f02-c141-e3f6-901e-f0c9222be423,292,4
7,46.249709,-119.100407,2018-04-08 08:14:07.000,c25r1g98p6,21191f02-c141-e3f6-901e-f0c9222be423,93,4
8,46.167117,-118.952341,2018-04-08 08:42:45.500,c25qthmwcw,21191f02-c141-e3f6-901e-f0c9222be423,78,4
9,46.138407,-118.801599,2018-04-08 09:11:17.000,c25w2x5rdn,21191f02-c141-e3f6-901e-f0c9222be423,241,4


In [43]:
import pandas as pd
from datetime import datetime, timedelta

def colocation(data: pd.DataFrame, lois: pd.DataFrame, duration: int) -> pd.DataFrame:
    """
    Finds devices that were detected within a certain distance and time range of a given set of locations of interest.

    Args:
        data: A DataFrame containing all of the data.
        lois: A DataFrame of locations of interest for a single advertiser ID.
        duration: An int representing the time range in hours to match timeframes of datapoints.

    Returns:
        A DataFrame containing co-located devices.
    """
    relevant_features = ['geohash', 'datetime', 'latitude', 'longitude', 'advertiser_id']
    data = data[relevant_features].sort_values(by="datetime").reset_index(drop=True)

    filtered = data[data.geohash.isin(lois.geohash.unique())]
    
    try:
        filtered = filtered[filtered.advertiser_id != lois.advertiser_id.iloc[0]]
    except:
        pass
    
    print(filtered)
    
    search_time = timedelta(hours=duration)
    
    def time_filter(row):
        loi_filtered = lois[lois.geohash == row.geohash]
        loi_dates = pd.to_datetime(loi_filtered.datetime)
        filtered_time = pd.to_datetime(row.datetime)
        within_timerange = (filtered_time > (loi_dates - search_time)) & (filtered_time < (loi_dates + search_time))
        row['remove'] = not within_timerange.any()
        return row

    filtered = filtered.apply(time_filter, axis=1)
    data_out = None
    if 'remove' in filtered.columns:
        data_out = filtered.loc[filtered.remove == False].drop(columns=['remove'])

    return data_out


In [64]:
loi_data = filter_close_coordinates(loi_data, threshold_distance=.5)
loi_data

/tmp/ipykernel_8669/2294503313.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row1, ignore_index=True)


,latitude,longitude,datetime,geohash,advertiser_id,label,i
0,46.138150,-118.772190,2018-04-08 09:15:57.500,c25w3r0wpq,21191f02-c141-e3f6-901e-f0c9222be423,296.0,1.0
1,46.212030,-119.017008,2018-04-08 08:29:47.500,c25qgswkwd,21191f02-c141-e3f6-901e-f0c9222be423,298.0,2.0
2,46.138390,-118.799321,2018-04-08 09:11:39.000,c25w2xhzx5,21191f02-c141-e3f6-901e-f0c9222be423,7.0,2.0
3,46.254613,-119.146024,2018-04-08 08:06:01.000,c25r0u2s8k,21191f02-c141-e3f6-901e-f0c9222be423,293.0,3.0
4,46.188440,-118.983064,2018-04-08 08:36:24.000,c25qu2r5jt,21191f02-c141-e3f6-901e-f0c9222be423,282.0,3.0
5,46.165614,-118.950202,2018-04-08 08:43:24.000,c25qthpn5q,21191f02-c141-e3f6-901e-f0c9222be423,300.0,4.0
6,46.185150,-118.978318,2018-04-08 08:37:44.500,c25qsxf8h2,21191f02-c141-e3f6-901e-f0c9222be423,243.0,4.0
7,46.219894,-119.028312,2018-04-08 08:27:49.500,c25qgqn4rk,21191f02-c141-e3f6-901e-f0c9222be423,46.0,4.0
8,46.138688,-118.841495,2018-04-08 09:03:54.000,c25qrxq326,21191f02-c141-e3f6-901e-f0c9222be423,30.0,4.0
9,46.277680,-119.194261,2018-04-08 14:24:29.000,c25pr8t7h2,21191f02-c141-e3f6-901e-f0c9222be423,NaN,NaN


In [65]:
# Create a Folium map centered around the mean coordinates
map_center = [loi_data['latitude'].mean(), loi_data['longitude'].mean()]
folium_map = folium.Map(location=map_center, zoom_start=5)

# Add markers for each row in the DataFrame
for index, row in loi_data.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']]).add_to(folium_map)
    
folium_map

In [44]:
colocation = colocation(df, loi_data, duration=12)
colocation

            geohash            datetime   latitude  longitude  \
976736   c25r15dhqq 2018-04-08 17:08:14  46.250460 -119.13272   
1111676  c25qsxdz13 2018-04-08 19:04:11  46.184988 -118.97810   
1210370  c25qu1xsws 2018-04-08 20:00:59  46.195562 -118.99332   
1308928  c25qsxdz13 2018-04-08 22:38:35  46.184988 -118.97810   

                                advertiser_id  
976736   83199917-33f3-6c46-f1e0-55b051ef2444  
1111676  720bb644-61a8-681c-b282-b7410370d38a  
1210370  3959d833-4cd4-e676-2331-1c5d55870264  
1308928  f7655f03-4219-28bd-04e0-2e6e97917085  


,geohash,datetime,latitude,longitude,advertiser_id
976736,c25r15dhqq,2018-04-08 17:08:14,46.250460,-119.13272,83199917-33f3-6c46-f1e0-55b051ef2444
1111676,c25qsxdz13,2018-04-08 19:04:11,46.184988,-118.97810,720bb644-61a8-681c-b282-b7410370d38a
1210370,c25qu1xsws,2018-04-08 20:00:59,46.195562,-118.99332,3959d833-4cd4-e676-2331-1c5d55870264


In [56]:
loi_data = pd.DataFrame()

for i in range(1,25,3):
    for j in range(73,0,-4):
        print(i,j)
        loi_data = pd.concat([loi_data,locations_of_interest(df, adid, i, j)]).reset_index(drop=True)
        
loi_data

1 73
1 69
1 65
1 61
1 57
1 53
1 49
1 45
1 41
1 37
1 33
1 29
1 25
1 21
1 17
1 13
1 9
1 5
4 73
4 69
4 65
4 61
4 57
4 53
4 49
4 45
4 41
4 37
4 33
4 29
4 25
4 21
4 17
4 13
4 9
4 5
7 73
7 69
7 65
7 61
7 57
7 53
7 49
7 45
7 41
7 37
7 33
7 29
7 25
7 21
7 17
7 13
7 9
7 5
10 73
10 69
10 65
10 61
10 57
10 53
10 49
10 45
10 41
10 37
10 33
10 29
10 25
10 21
10 17
10 13
10 9
10 5
13 73
13 69
13 65
13 61
13 57
13 53
13 49
13 45
13 41
13 37
13 33
13 29
13 25
13 21
13 17
13 13
13 9
13 5
16 73
16 69
16 65
16 61
16 57
16 53
16 49
16 45
16 41
16 37
16 33
16 29
16 25
16 21
16 17
16 13
16 9
16 5
19 73
19 69
19 65
19 61
19 57
19 53
19 49
19 45
19 41
19 37
19 33
19 29
19 25
19 21
19 17
19 13
19 9
19 5
22 73
22 69
22 65
22 61
22 57
22 53
22 49
22 45
22 41
22 37
22 33
22 29
22 25
22 21
22 17
22 13
22 9
22 5


,geohash,datetime,latitude,longitude,advertiser_id
0,c25pr8t7h2,2018-04-08 14:24:29,46.277680,-119.194261,21191f02-c141-e3f6-901e-f0c9222be423
1,c25r50813j,2018-04-08 14:50:22,46.233462,-119.047800,21191f02-c141-e3f6-901e-f0c9222be423
2,c25pr8t7h2,2018-04-08 14:24:29,46.277680,-119.194261,21191f02-c141-e3f6-901e-f0c9222be423
3,c25r50813j,2018-04-08 14:50:22,46.233462,-119.047800,21191f02-c141-e3f6-901e-f0c9222be423
4,c25pr8t7h2,2018-04-08 14:24:29,46.277680,-119.194261,21191f02-c141-e3f6-901e-f0c9222be423
5,c25r50813j,2018-04-08 14:50:22,46.233462,-119.047800,21191f02-c141-e3f6-901e-f0c9222be423
6,c25pr8t7h2,2018-04-08 14:24:29,46.277680,-119.194261,21191f02-c141-e3f6-901e-f0c9222be423
7,c25r50813j,2018-04-08 14:50:22,46.233462,-119.047800,21191f02-c141-e3f6-901e-f0c9222be423
8,c25pr8t7h2,2018-04-08 14:24:29,46.277680,-119.194261,21191f02-c141-e3f6-901e-f0c9222be423
9,c25r50813j,2018-04-08 14:50:22,46.233462,-119.047800,21191f02-c141-e3f6-901e-f0c9222be423


In [57]:
loi_data.drop_duplicates()

,geohash,datetime,latitude,longitude,advertiser_id
0,c25pr8t7h2,2018-04-08 14:24:29,46.277680,-119.194261,21191f02-c141-e3f6-901e-f0c9222be423
1,c25r50813j,2018-04-08 14:50:22,46.233462,-119.047800,21191f02-c141-e3f6-901e-f0c9222be423
